# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df_0 = pd.read_csv('D:/geo_data_0.csv')
df_1 = pd.read_csv('D:/geo_data_1.csv')
df_2 = pd.read_csv('D:/geo_data_2.csv')

features_0 = df_0.drop(['product', 'id'], axis=1)
target_0 = df_0['product']
features_1 = df_1.drop(['product', 'id'], axis=1)
target_1 = df_1['product']
features_2 = df_2.drop(['product', 'id'], axis=1)
target_2 = df_2['product']

state = np.random.RandomState(0)
'''
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=state)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=state)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=state)
'''

'\nfeatures_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=state)\nfeatures_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=state)\nfeatures_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=state)\n'

## Обучение и проверка модели

In [59]:
model = LinearRegression()

def oil_predict(df, target_name):
    features = df.drop([target_name, 'id'] , axis=1)
    target = df[target_name]
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=state)
    
    model.fit(features_train, target_train)
    predicted_target = model.predict(features_valid)
    data = {'real_product' : target_valid, 'predicted_product' : predicted_target}
    data_frame = pd.DataFrame(data, columns = ['real_product', 'predicted_product'])
    rmse = mean_squared_error(target_valid, predicted_target) ** 0.5
    mean = data_frame['predicted_product'].mean()
    oil_sum = data_frame['predicted_product'].sum()
    return data_frame, rmse, mean, oil_sum

df_0_predicted, df_0_rmse, df_0_predicted_mean, df_0_predicted_sum = oil_predict(df_0, 'product')
print(df_0_predicted.head())
print('RMSE -', df_0_rmse)
print('Средний запас запасанного сырья -', df_0_predicted_mean)

df_1_predicted, df_1_rmse, df_1_predicted_mean, df_1_predicted_sum = oil_predict(df_1, 'product')
print(df_1_predicted.head())
print('RMSE -', df_1_rmse)
print('Средний запас запасанного сырья -', df_1_predicted_mean)

df_2_predicted, df_2_rmse, df_2_predicted_mean, df_2_predicted_sum = oil_predict(df_2, 'product')
print(df_2_predicted.head())
print('RMSE -', df_2_rmse)
print('Средний запас запасанного сырья -', df_2_predicted_mean)
'''
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predict_0 = model_0.predict(features_valid_0)
rmse_0 = mean_squared_error(target_valid_0, predict_0) ** 0.5
print('mean product prediction region 0:', predict_0.mean())
print('rmse region 0:', rmse_0)

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predict_1 = model_1.predict(features_valid_1)
rmse_1 = mean_squared_error(target_valid_1, predict_1) ** 0.5
print('mean product prediction region 1:', predict_1.mean())
print('rmse region 1:', rmse_1)

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predict_2 = model_2.predict(features_valid_2)
rmse_2 = mean_squared_error(target_valid_2, predict_2) ** 0.5
print('mean product prediction region 2:', predict_2.mean())
print('rmse region 2:', rmse_2)

# самый высокий показатель предсказанного количества сырья в регионе '2', но и отклонение там также самое высокое
# RMSE региона '1' очень низко из-за очень часто повторяющихся значений, среднее предсказанное сырья самое низкое среди всех
# регион '0' очень схож с регионом '2'
'''

       real_product  predicted_product
3582      49.723143          95.303891
60498     10.583795          95.098407
53227    139.110613         106.702273
21333     85.109875          96.740059
3885      39.726827          97.703381
RMSE - 37.48100896950594
Средний запас запасанного сырья - 92.27144852242301
       real_product  predicted_product
10655    107.813044         107.659247
29762     84.038886          82.623658
63784      3.179103           3.708318
10594     80.859783          81.229758
20934      0.000000           0.687698
RMSE - 0.89570180076533
Средний запас запасанного сырья - 69.02446949104798
       real_product  predicted_product
16037      1.525959          97.886113
17477     39.747977          53.709118
92520     94.979266         110.286626
48861    126.430363         114.525841
47878    141.620720          83.728522
RMSE - 39.98689683154368
Средний запас запасанного сырья - 94.903381366729


"\nmodel_0 = LinearRegression()\nmodel_0.fit(features_train_0, target_train_0)\npredict_0 = model_0.predict(features_valid_0)\nrmse_0 = mean_squared_error(target_valid_0, predict_0) ** 0.5\nprint('mean product prediction region 0:', predict_0.mean())\nprint('rmse region 0:', rmse_0)\n\nmodel_1 = LinearRegression()\nmodel_1.fit(features_train_1, target_train_1)\npredict_1 = model_1.predict(features_valid_1)\nrmse_1 = mean_squared_error(target_valid_1, predict_1) ** 0.5\nprint('mean product prediction region 1:', predict_1.mean())\nprint('rmse region 1:', rmse_1)\n\nmodel_2 = LinearRegression()\nmodel_2.fit(features_train_2, target_train_2)\npredict_2 = model_2.predict(features_valid_2)\nrmse_2 = mean_squared_error(target_valid_2, predict_2) ** 0.5\nprint('mean product prediction region 2:', predict_2.mean())\nprint('rmse region 2:', rmse_2)\n\n# самый высокий показатель предсказанного количества сырья в регионе '2', но и отклонение там также самое высокое\n# RMSE региона '1' очень низко

## Подготовка к расчёту прибыли

In [60]:
budget = 1e+07 #Мы расчитываем прибыль в тысячах рублей за тысячу барелей, поэтому 1 милиард сохранен таким образом
chosen_points = 500
best_points = 200
price_per_barrel = 450
average_profit_per_point = budget / best_points
print('Средняя прибыль на лучшую точку -', average_profit_per_point, 'тыс.руб.')

average_barrels_per_point = average_profit_per_point  / price_per_barrel
print('Средний объем нефти на точку -',average_barrels_per_point, 'тыс. барелей')

average_barrels = budget / price_per_barrel
print('Средний объем нефти на регион -',average_barrels, 'тыс. барелей')

Средняя прибыль на лучшую точку - 50000.0 тыс.руб.
Средний объем нефти на точку - 111.11111111111111 тыс. барелей
Средний объем нефти на регион - 22222.222222222223 тыс. барелей


## Расчёт прибыли и рисков 

In [61]:
predict_0 = pd.Series(data=predict_0)
predict_1 = pd.Series(data=predict_1)
predict_2 = pd.Series(data=predict_2)

def oil_profit_calculation(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return price_per_barrel * selected.sum() - budget

state = np.random.RandomState(5062020)
def bootstrap_1000(target, probs):
    values=[]
    lost = 0
    for i in range(1000):
        target_sample = target.sample(replace=True, random_state=state, n=chosen_points)
        probs_sample = probs[target_sample.index]
        profit = oil_profit_calculation(target, probs_sample, best_points)
        if profit < 0:
            lost +=1
        values.append(profit)
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    average = values.mean()
    risk = lost / 1000
    return average, lower, upper, risk
'''
bootstrap_1000(df_0_predicted['real_product'], df_0_predicted['predicted_product'])
bootstrap_1000(df_1_predicted['real_product'], df_1_predicted['predicted_product'])
bootstrap_1000(df_2_predicted['real_product'], df_2_predicted['predicted_product'])
'''
average_0, lower_0, upper_0, risk_0 = bootstrap_1000(df_0_predicted['real_product'], df_0_predicted['predicted_product'])
print('Данные по первому региону.')
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(average_0))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_0, upper_0))
print('Процент риска {:.2%}'.format(risk_0))

average_1, lower_1, upper_1, risk_1 = bootstrap_1000(df_1_predicted['real_product'], df_1_predicted['predicted_product'])
print('Данные по первому региону.')
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(average_1))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_1, upper_1))
print('Процент риска {:.2%}'.format(risk_1))

average_2, lower_2, upper_2, risk_2 = bootstrap_1000(df_2_predicted['real_product'], df_2_predicted['predicted_product'])
print('Данные по первому региону.')
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(average_2))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_2, upper_2))
print('Процент риска {:.2%}'.format(risk_2))
# commit git test

Данные по первому региону.
Средняя прибыль по региону = 454086.36 тыс.руб.
95% доверительный интервал от -94757.43 до 968007.32 тыс.руб.
Процент риска 5.30%
Данные по первому региону.
Средняя прибыль по региону = 459524.07 тыс.руб.
95% доверительный интервал от 62332.91 до 879417.91 тыс.руб.
Процент риска 1.50%
Данные по первому региону.
Средняя прибыль по региону = 419226.80 тыс.руб.
95% доверительный интервал от -103901.35 до 927579.81 тыс.руб.
Процент риска 5.20%


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован